# 1.**Manage Gpu**


In [ ]:
!nvidia-smi

Sat Jun  4 05:07:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **2.connectin Colab and Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 
 ##make a shortlink for our address
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# 3.install and setting Darknet

In [ ]:
#clone Darknet
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
#configuration Darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
#compile Darknet
!make

In [ ]:
!ls

## 4.setting yolo

In [ ]:
#Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_traning.cfg

In [ ]:
#change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_traning.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_traning.cfg
!sed -i 's/max_batches = 500200/max_batches=6000/' cfg/yolov3_traning.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_traning.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_traning.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_traning.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_traning.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_traning.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_traning.cfg



# 5.make file and save result


In [ ]:
#create .names files
!echo -e 'Waring Mask\nNot wearing Mask' > data/obj.names


In [ ]:
# creat .data files
!echo -e 'classes= 2\ntrain = data/train.txt\n valid = data/test.txt\n names = data/obj.names\n backup=/mydrive/yolov3' > data/obj.data

# `6.give image path and label`


In [ ]:
#create a folder and unzip image dataset
!mkdir data/obj
!unzip /mydrive/yolov3/archive.zip -d data/obj
!unzip /mydrive/yolov3/images.zip -d data/Tobj


In [ ]:
#Create train.txt file
import glob
images_list = glob.glob("data/obj/archive/*.jpg")
print(images_list)
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

    


[]


In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)
with open("data/test.txt", "w") as f:
    f.write("\n".join(images_list))

    

[]


# 7.Transfer learning

In [ ]:
# Download pre-trained weights for the convolutional layers file
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
# Start training

!./darknet detector train data/obj.data  cfg/yolov3_traning.cfg  darknet53.conv.74 -dont_show -mjpeg_port 8090 -map
#!./darknet detector train data/obj.data cfg/yolov3_traning.cfg   ../gdrive/MyDrive/yolov3/yolov3_traning_last.weights -dont_show 